# Results:



# Purpose:

2015-04-02 (Thursday)

Create code to make tables with info from `argot2` annotations for all genes in a gene-list.

# Shortcuts:

[Convert-lines-from-in_files-to-BED-type](#Convert-lines-from-in_files-to-BED-type)

[Prepare-and-run-the-Bedtools-window-analysis-commands](#Prepare-and-run-the-Bedtools-window-analysis-commands)

[Set-up-annotation-database:](#Set-up-annotation-database:)

[Set-up-gene/SNP-relationship-data:](#Set-up-gene/SNP-relationship-data:)

[Create-tables-for-our-genes-that-reference-the-SNP-location-they-were-associated-with:](#Create-tables-for-our-genes-that-reference-the-SNP-location-they-were-associated-with:)

[Functional-Annotation-of-the-genes-associated-with-the-top-5-environment-SNPs:](#Functional-Annotation-of-the-genes-associated-with-the-top-5-environment-SNPs:)

[Answer-set-1](#Answer-set-1)

## Imports:

In [167]:
# imports
import csv

from collections import defaultdict

import pandas as pd
pd.set_option('display.max_columns', 60)

import munch

from IPython.display import display, HTML

import pybedtools as pbt

import sh

import tabulate
tbl = tabulate.tabulate

from spartan.utils.annotations.ensembl.gff3 import parse_gff3
from spartan.utils.annotations.ensembl.gff3 import parse_gff3_attributes
from spartan.utils.files import tableFile2namedTuple

from spartan.utils.genome_specific.GfusI1 import GfusI1_0

## File paths:

In [6]:
# File Paths

## Functional annatation (Argot2)
fanno = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/functional/GfusI1.1_pre/argot2_out/argot_functional_annotations_ts150.h5"

## For setting up the BEDTOOLS phase
btools_out_dir = "/home/gus/remote_mounts/louise/data/projects/ddrad58/SNPs_of_interest/2015_04_03_genes_near_SNPs"
btools_gene_models_gff3 = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/annotations/Glossina-fuscipes-IAEA_BASEFEATURES_GfusI1.1.gff3"




## SNPs of interest CSV [ IN FILES ] ----------------------------------------------------------

#### from folder: 2015-03-17_env_selection
Selected_PopPairwiseMSNB_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSNB_Environm.csv"
Selected_PopPairwiseMSOT_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSOT_Environm.csv"
Selected_PopPairwiseMSvsNBOT_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSvsNBOT_Environm.csv"

#### from folder: 2015-02-26_bayescan_tables 

Top10_PopPairwiseOverlap_Infection = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Infection.csv"
Top10_PopPairwiseOverlap_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Environm.csv"
Top05_InfectionOverall = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_InfectionOverall.csv"
Top05_PopPairwiseOverlap_Environm = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_PopPairwiseOverlap_Environm.csv"
Top05_PopPairwiseOverlap_Infection = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_PopPairwiseOverlap_Infection.csv"
Top10_InfectionOverall = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_InfectionOverall.csv"


## SNPs of interest BED [ OUT FILES ] ----------------------------------------------------------

#### from folder: 2015-03-17_env_selection
Selected_PopPairwiseMSNB_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSNB_Environm.bed"
Selected_PopPairwiseMSOT_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSOT_Environm.bed"
Selected_PopPairwiseMSvsNBOT_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection/Selected_PopPairwiseMSvsNBOT_Environm.bed"

#### from folder: 2015-02-26_bayescan_tables
Top10_PopPairwiseOverlap_Infection_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Infection.bed"
Top10_PopPairwiseOverlap_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_PopPairwiseOverlap_Environm.bed"
Top05_InfectionOverall_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_InfectionOverall.bed"
Top05_PopPairwiseOverlap_Environm_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_PopPairwiseOverlap_Environm.bed"
Top05_PopPairwiseOverlap_Infection_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top05_PopPairwiseOverlap_Infection.bed"
Top10_InfectionOverall_bed = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-02-26_bayescan_tables/Top10_InfectionOverall.bed"


## Fasta file for renaming contigs ----------------------------------------------------------
fasta = "/home/gus/remote_mounts/louise/data/genomes/glossina_fuscipes/assemblies/GfusI1/Glossina-fuscipes-IAEA_SCAFFOLDS_GfusI1.fa"

## outfiles for windowBed analysis ----------------------------------------------------------
window_bed_dir = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/ddrad58/data_from_andrea/2015-03-17_env_selection"

In [7]:
def make_BED_line(in_line, name_map):
    chrom = name_map[in_line[1]]
    chromstart = str(int(in_line[2]) - 1)
    chromend = str(int(in_line[2]))
    
    return chrom, chromstart, chromend

def get_in_lines(path, skip_first_row=True):
    with open(path, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        if skip_first_row:
            reader.next()
        for line in reader:
            yield line
            
def write_line(out_file, line):
    line = "%s\n" % ('\t'.join(line))
    out_file.write(line)

def convert_a_file(in_path, out_path, name_map):
    with open(out_path, 'wb') as out_file:
        
        in_lines = get_in_lines(in_path)

        for line in in_lines:
            bed_data = make_BED_line(line, name_map)
            write_line(out_file,bed_data)


# Convert lines from `in_files` to BED type 

In [8]:
name_map = GfusI1_0.get_name_map_from_fasta_headers(fasta)
name_map['KK352174.1']

'Scaffold391'

### _NOTE: I had to do a bit of manual editing of the input files_

1. clean the scaffold name to JUST the supercontig info
2. delete the \*extra\*  column at the bigining of each data line (I think it was the `R` index)

In [9]:
!head $Selected_PopPairwiseMSNB_Environm

In [10]:
convert_a_file(Selected_PopPairwiseMSNB_Environm, Selected_PopPairwiseMSNB_Environm_bed, name_map)

In [11]:
!head $Selected_PopPairwiseMSNB_Environm_bed

Scaffold8	1482580	1482581
Scaffold85	94212	94213
Scaffold107	378348	378349
Scaffold206	259638	259639
Scaffold427	13827	13828
Scaffold23	1150376	1150377
Scaffold69	308923	308924
Scaffold93	365365	365366
Scaffold93	365391	365392
Scaffold144	334226	334227


## Do conversions for remander of files:

In [12]:
convert_a_file(Selected_PopPairwiseMSOT_Environm,
               Selected_PopPairwiseMSOT_Environm_bed,
               name_map)

convert_a_file(Selected_PopPairwiseMSvsNBOT_Environm,
               Selected_PopPairwiseMSvsNBOT_Environm_bed,
               name_map)

convert_a_file(Top10_PopPairwiseOverlap_Infection,
               Top10_PopPairwiseOverlap_Infection_bed,
               name_map)

convert_a_file(Top10_PopPairwiseOverlap_Environm,
               Top10_PopPairwiseOverlap_Environm_bed,
               name_map)

convert_a_file(Top05_InfectionOverall,
               Top05_InfectionOverall_bed,name_map)

convert_a_file(Top05_PopPairwiseOverlap_Environm,
               Top05_PopPairwiseOverlap_Environm_bed,
               name_map)

convert_a_file(Top05_PopPairwiseOverlap_Infection,
               Top05_PopPairwiseOverlap_Infection_bed,
               name_map)

convert_a_file(Top10_InfectionOverall, 
               Top10_InfectionOverall_bed,
               name_map)

# Prepare and run the `Bedtools window` analysis commands

In [13]:
# load bed files into pybedtools objects
Selected_PopPairwiseMSNB_Environm_pbt = pbt.BedTool(Selected_PopPairwiseMSNB_Environm_bed)
Selected_PopPairwiseMSOT_Environm_pbt = pbt.BedTool(Selected_PopPairwiseMSOT_Environm_bed)
Selected_PopPairwiseMSvsNBOT_Environm_pbt = pbt.BedTool(Selected_PopPairwiseMSvsNBOT_Environm_bed)
Top10_PopPairwiseOverlap_Infection_pbt = pbt.BedTool(Top10_PopPairwiseOverlap_Infection_bed)
Top10_PopPairwiseOverlap_Environm_pbt = pbt.BedTool(Top10_PopPairwiseOverlap_Environm_bed)
Top05_InfectionOverall_pbt = pbt.BedTool(Top05_InfectionOverall_bed)
Top05_PopPairwiseOverlap_Environm_pbt = pbt.BedTool(Top05_PopPairwiseOverlap_Environm_bed)
Top05_PopPairwiseOverlap_Infection_pbt = pbt.BedTool(Top05_PopPairwiseOverlap_Infection_bed)
Top10_InfectionOverall_pbt = pbt.BedTool(Top10_InfectionOverall_bed)

# load gene models into pybedtools object and filter for only gene features
btools_gene_models_pbt = pbt.BedTool(btools_gene_models_gff3)
btools_gene_models_pbt_genes = btools_gene_models_pbt.filter(lambda x: x[2] == 'gene').saveas()

In [14]:
# run the windowBed calls

# window == 500
Selected_PopPairwiseMSNB_Environm_win500 = Selected_PopPairwiseMSNB_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Selected_PopPairwiseMSOT_Environm_win500 = Selected_PopPairwiseMSOT_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Selected_PopPairwiseMSvsNBOT_Environm_win500 = Selected_PopPairwiseMSvsNBOT_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Top10_PopPairwiseOverlap_Infection_win500 = Top10_PopPairwiseOverlap_Infection_pbt.window(btools_gene_models_pbt_genes, w=500)
Top10_PopPairwiseOverlap_Environm_win500 = Top10_PopPairwiseOverlap_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Top05_InfectionOverall_win500 = Top05_InfectionOverall_pbt.window(btools_gene_models_pbt_genes, w=500)
Top05_PopPairwiseOverlap_Environm_win500 = Top05_PopPairwiseOverlap_Environm_pbt.window(btools_gene_models_pbt_genes, w=500)
Top05_PopPairwiseOverlap_Infection_win500 = Top05_PopPairwiseOverlap_Infection_pbt.window(btools_gene_models_pbt_genes, w=500)
Top10_InfectionOverall_win500 = Top10_InfectionOverall_pbt.window(btools_gene_models_pbt_genes, w=500)

# window == 1000
Selected_PopPairwiseMSNB_Environm_win1000 = Selected_PopPairwiseMSNB_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Selected_PopPairwiseMSOT_Environm_win1000 = Selected_PopPairwiseMSOT_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Selected_PopPairwiseMSvsNBOT_Environm_win1000 = Selected_PopPairwiseMSvsNBOT_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top10_PopPairwiseOverlap_Infection_win1000 = Top10_PopPairwiseOverlap_Infection_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top10_PopPairwiseOverlap_Environm_win1000 = Top10_PopPairwiseOverlap_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top05_InfectionOverall_win1000 = Top05_InfectionOverall_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top05_PopPairwiseOverlap_Environm_win1000 = Top05_PopPairwiseOverlap_Environm_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top05_PopPairwiseOverlap_Infection_win1000 = Top05_PopPairwiseOverlap_Infection_pbt.window(btools_gene_models_pbt_genes, w=1000)
Top10_InfectionOverall_win1000 = Top10_InfectionOverall_pbt.window(btools_gene_models_pbt_genes, w=1000)

In [15]:
# save the output

Selected_PopPairwiseMSNB_Environm_win500 = Selected_PopPairwiseMSNB_Environm_win500.moveto("{win_dir}/Selected_PopPairwiseMSNB_Environm_win500.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSOT_Environm_win500 = Selected_PopPairwiseMSOT_Environm_win500.moveto("{win_dir}/Selected_PopPairwiseMSOT_Environm_win500.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSvsNBOT_Environm_win500 = Selected_PopPairwiseMSvsNBOT_Environm_win500.moveto("{win_dir}/Selected_PopPairwiseMSvsNBOT_Environm_win500.bed".format(win_dir=window_bed_dir))
Top10_PopPairwiseOverlap_Infection_win500 = Top10_PopPairwiseOverlap_Infection_win500.moveto("{win_dir}/Top10_PopPairwiseOverlap_Infection_win500.bed".format(win_dir=window_bed_dir))
Top10_PopPairwiseOverlap_Environm_win500 = Top10_PopPairwiseOverlap_Environm_win500.moveto("{win_dir}/Top10_PopPairwiseOverlap_Environm_win500.bed".format(win_dir=window_bed_dir))
Top05_InfectionOverall_win500 = Top05_InfectionOverall_win500.moveto("{win_dir}/Top05_InfectionOverall_win500.bed".format(win_dir=window_bed_dir))
Top05_PopPairwiseOverlap_Environm_win500 = Top05_PopPairwiseOverlap_Environm_win500.moveto("{win_dir}/Top05_PopPairwiseOverlap_Environm_win500.bed".format(win_dir=window_bed_dir))
Top05_PopPairwiseOverlap_Infection_win500 = Top05_PopPairwiseOverlap_Infection_win500.moveto("{win_dir}/Top05_PopPairwiseOverlap_Infection_win500.bed".format(win_dir=window_bed_dir))
Top10_InfectionOverall_win500 = Top10_InfectionOverall_win500.moveto("{win_dir}/Top10_InfectionOverall_win500.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSNB_Environm_win1000 = Selected_PopPairwiseMSNB_Environm_win1000.moveto("{win_dir}/Selected_PopPairwiseMSNB_Environm_win1000.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSOT_Environm_win1000 = Selected_PopPairwiseMSOT_Environm_win1000.moveto("{win_dir}/Selected_PopPairwiseMSOT_Environm_win1000.bed".format(win_dir=window_bed_dir))
Selected_PopPairwiseMSvsNBOT_Environm_win1000 = Selected_PopPairwiseMSvsNBOT_Environm_win1000.moveto("{win_dir}/Selected_PopPairwiseMSvsNBOT_Environm_win1000.bed".format(win_dir=window_bed_dir))
Top10_PopPairwiseOverlap_Infection_win1000 = Top10_PopPairwiseOverlap_Infection_win1000.moveto("{win_dir}/Top10_PopPairwiseOverlap_Infection_win1000.bed".format(win_dir=window_bed_dir))
Top10_PopPairwiseOverlap_Environm_win1000 = Top10_PopPairwiseOverlap_Environm_win1000.moveto("{win_dir}/Top10_PopPairwiseOverlap_Environm_win1000.bed".format(win_dir=window_bed_dir))
Top05_InfectionOverall_win1000 = Top05_InfectionOverall_win1000.moveto("{win_dir}/Top05_InfectionOverall_win1000.bed".format(win_dir=window_bed_dir))
Top05_PopPairwiseOverlap_Environm_win1000 = Top05_PopPairwiseOverlap_Environm_win1000.moveto("{win_dir}/Top05_PopPairwiseOverlap_Environm_win1000.bed".format(win_dir=window_bed_dir))
Top05_PopPairwiseOverlap_Infection_win1000 = Top05_PopPairwiseOverlap_Infection_win1000.moveto("{win_dir}/Top05_PopPairwiseOverlap_Infection_win1000.bed".format(win_dir=window_bed_dir))
Top10_InfectionOverall_win1000 = Top10_InfectionOverall_win1000.moveto("{win_dir}/Top10_InfectionOverall_win1000.bed".format(win_dir=window_bed_dir))

# Set up main data variables:

### Set up annotation database:

In [16]:
argot2 = pd.read_hdf(path_or_buf=fanno, key='dataframe')

In [154]:
argot2['gene_id'] = argot2.Sequence.apply(lambda x: x[:-3])
argot2_200 = argot2[argot2['Total Score'] >= 200]
argot2.head()

,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content,gene_id
0,GFUI034947-PA,P,GO:0006508,proteolysis,270.313447,0.496543,8.247696,GFUI034947
1,GFUI035874-PA,F,GO:0005515,protein binding,529.038456,0.500000,5.471582,GFUI035874
2,GFUI033625-PA,P,GO:0005980,glycogen catabolic process,307.758251,0.284597,13.424450,GFUI033625
3,GFUI033625-PA,F,GO:0004134,4-alpha-glucanotransferase activity,159.513252,0.177063,10.153643,GFUI033625
4,GFUI033625-PA,F,GO:0004135,"amylo-alpha-1,6-glucosidase activity",184.352303,0.177063,11.734746,GFUI033625


### Set up gene/SNP relationship data:

#### Function to create dictionary-based retrieval object for gene/SNP data:

In [18]:
def bed3_v_gff3_window_by_gff3_ID(bedtools_window_out):
    """
    Returns dictionary-based retrieval object for Bedtools result from 3-field BED format records vs
    GFF3 format records data grouped by the GFF3 features.
    """
    headers = ["bed3_seq",
               "bed3_start",
               "bed3_end",
               "gff3_seq",
               "gff3_source",
               "gff3_type",
               "gff3_start",
               "gff3_end",
               "gff3_score",
               "gff3_strand",
               "gff3_phase",
               "gff3_attributes",]
    
    table = tableFile2namedTuple(bedtools_window_out, 
                                 sep='\t', 
                                 headers=headers)
    
    Tree = lambda: defaultdict(Tree)
    data = Tree()
    
    for row in table:
        gff3_rec = parse_gff3_attributes(row.gff3_attributes)['ID']

        data[gff3_rec]['info'].setdefault('seq', row.gff3_seq)
        data[gff3_rec]['info'].setdefault('source', row.gff3_source)
        data[gff3_rec]['info'].setdefault('type', row.gff3_type)
        data[gff3_rec]['info'].setdefault('start', row.gff3_start)
        data[gff3_rec]['info'].setdefault('end', row.gff3_end)
        data[gff3_rec]['info'].setdefault('score', row.gff3_score)
        data[gff3_rec]['info'].setdefault('strand', row.gff3_strand)
        data[gff3_rec]['info'].setdefault('phase', row.gff3_phase)
        data[gff3_rec]['info'].setdefault('attributes', row.gff3_attributes)

        bed3_hit = dict(seq = row.bed3_seq, 
                        start = row.bed3_start,
                        end = row.bed3_end)

        try:
            data[gff3_rec]['bed3_hits'].append(bed3_hit)

        except AttributeError:
            data[gff3_rec]['bed3_hits'] = []
            data[gff3_rec]['bed3_hits'].append(bed3_hit)

    return munch.munchify(data)

In [62]:
def snp_vs_gff_to_DF(bedtools_out):
    headers = ["bed3_seq",
               "bed3_start",
               "bed3_end",
               "gff3_seq",
               "gff3_source",
               "gff3_type",
               "gff3_start",
               "gff3_end",
               "gff3_score",
               "gff3_strand",
               "gff3_phase",
               "gff3_attributes",]
    df = pd.read_csv(bedtools_out, sep='\t', names=headers)
    
    gene_id = lambda x: parse_gff3_attributes(x)['ID']
    
    df['gff3_rec'] = df.gff3_attributes.apply(gene_id)
    
    return df

In [63]:
Selected_PopPairwiseMSNB_Environm_win500_search = snp_vs_gff_to_DF(Selected_PopPairwiseMSNB_Environm_win500.fn)
Selected_PopPairwiseMSOT_Environm_win500_search = snp_vs_gff_to_DF(Selected_PopPairwiseMSOT_Environm_win500.fn)
Selected_PopPairwiseMSvsNBOT_Environm_win500_search = snp_vs_gff_to_DF(Selected_PopPairwiseMSvsNBOT_Environm_win500.fn)
Top10_PopPairwiseOverlap_Infection_win500_search = snp_vs_gff_to_DF(Top10_PopPairwiseOverlap_Infection_win500.fn)
Top10_PopPairwiseOverlap_Environm_win500_search = snp_vs_gff_to_DF(Top10_PopPairwiseOverlap_Environm_win500.fn)
Top05_InfectionOverall_win500_search = snp_vs_gff_to_DF(Top05_InfectionOverall_win500.fn)
Top05_PopPairwiseOverlap_Environm_win500_search = snp_vs_gff_to_DF(Top05_PopPairwiseOverlap_Environm_win500.fn)
Top05_PopPairwiseOverlap_Infection_win500_search = snp_vs_gff_to_DF(Top05_PopPairwiseOverlap_Infection_win500.fn)
Top10_InfectionOverall_win500_search = snp_vs_gff_to_DF(Top10_InfectionOverall_win500.fn)
Selected_PopPairwiseMSNB_Environm_win1000_search = snp_vs_gff_to_DF(Selected_PopPairwiseMSNB_Environm_win1000.fn)
Selected_PopPairwiseMSOT_Environm_win1000_search = snp_vs_gff_to_DF(Selected_PopPairwiseMSOT_Environm_win1000.fn)
Selected_PopPairwiseMSvsNBOT_Environm_win1000_search = snp_vs_gff_to_DF(Selected_PopPairwiseMSvsNBOT_Environm_win1000.fn)
Top10_PopPairwiseOverlap_Infection_win1000_search = snp_vs_gff_to_DF(Top10_PopPairwiseOverlap_Infection_win1000.fn)
Top10_PopPairwiseOverlap_Environm_win1000_search = snp_vs_gff_to_DF(Top10_PopPairwiseOverlap_Environm_win1000.fn)
Top05_InfectionOverall_win1000_search = snp_vs_gff_to_DF(Top05_InfectionOverall_win1000.fn)
Top05_PopPairwiseOverlap_Environm_win1000_search = snp_vs_gff_to_DF(Top05_PopPairwiseOverlap_Environm_win1000.fn)
Top05_PopPairwiseOverlap_Infection_win1000_search = snp_vs_gff_to_DF(Top05_PopPairwiseOverlap_Infection_win1000.fn)
Top10_InfectionOverall_win1000_search = snp_vs_gff_to_DF(Top10_InfectionOverall_win1000.fn)

In [78]:
fr = func_results = munch.Munch(
Selected_PopPairwiseMSNB_Environm_win500=Selected_PopPairwiseMSNB_Environm_win500_search,
Selected_PopPairwiseMSOT_Environm_win500=Selected_PopPairwiseMSOT_Environm_win500_search,
Selected_PopPairwiseMSvsNBOT_Environm_win500=Selected_PopPairwiseMSvsNBOT_Environm_win500_search,
Top10_PopPairwiseOverlap_Infection_win500=Top10_PopPairwiseOverlap_Infection_win500_search,
Top10_PopPairwiseOverlap_Environm_win500=Top10_PopPairwiseOverlap_Environm_win500_search,
Top05_InfectionOverall_win500=Top05_InfectionOverall_win500_search,
Top05_PopPairwiseOverlap_Environm_win500=Top05_PopPairwiseOverlap_Environm_win500_search,
Top05_PopPairwiseOverlap_Infection_win500=Top05_PopPairwiseOverlap_Infection_win500_search,
Top10_InfectionOverall_win500=Top10_InfectionOverall_win500_search,
Selected_PopPairwiseMSNB_Environm_win1000=Selected_PopPairwiseMSNB_Environm_win1000_search,
Selected_PopPairwiseMSOT_Environm_win1000=Selected_PopPairwiseMSOT_Environm_win1000_search,
Selected_PopPairwiseMSvsNBOT_Environm_win1000=Selected_PopPairwiseMSvsNBOT_Environm_win1000_search,
Top10_PopPairwiseOverlap_Infection_win1000=Top10_PopPairwiseOverlap_Infection_win1000_search,
Top10_PopPairwiseOverlap_Environm_win1000=Top10_PopPairwiseOverlap_Environm_win1000_search,
Top05_InfectionOverall_win1000=Top05_InfectionOverall_win1000_search,
Top05_PopPairwiseOverlap_Environm_win1000=Top05_PopPairwiseOverlap_Environm_win1000_search,
Top05_PopPairwiseOverlap_Infection_win1000=Top05_PopPairwiseOverlap_Infection_win1000_search,
Top10_InfectionOverall_win1000=Top10_InfectionOverall_win1000_search
)

In [81]:
len(fr.Top10_PopPairwiseOverlap_Infection_win1000.merge(right=argot2, 
                                                      how='inner', 
                                                      on=None, 
                                                      left_on='gff3_rec', right_on='gene_id').gene_id.unique())

149

In [83]:
fr.Top10_PopPairwiseOverlap_Infection_win1000.merge(right=argot2, 
                                                      how='inner', 
                                                      on=None, 
                                                      left_on='gff3_rec', right_on='gene_id')

,bed3_seq,bed3_start,bed3_end,gff3_seq,gff3_source,gff3_type,gff3_start,gff3_end,gff3_score,gff3_strand,gff3_phase,gff3_attributes,gff3_rec,Sequence,Aspect,GO ID,Name,Total Score,Internal Confidence,Information Content,gene_id
0,Scaffold330,306455,306456,Scaffold330,VectorBase,gene,293458,313446,.,+,.,ID=GFUI025974;biotype=protein_coding,GFUI025974,GFUI025974-PA,F,GO:0005524,ATP binding,1183.663313,0.136078,4.006505,GFUI025974
1,Scaffold330,306455,306456,Scaffold330,VectorBase,gene,293458,313446,.,+,.,ID=GFUI025974;biotype=protein_coding,GFUI025974,GFUI025974-PA,F,GO:0003677,DNA binding,1437.680250,0.156676,3.831010,GFUI025974
2,Scaffold330,306455,306456,Scaffold330,VectorBase,gene,293458,313446,.,+,.,ID=GFUI025974;biotype=protein_coding,GFUI025974,GFUI025974-PA,F,GO:0016817,"hydrolase activity, acting on acid anhydrides",733.312642,0.298775,3.904510,GFUI025974
3,Scaffold330,306455,306456,Scaffold330,VectorBase,gene,293458,313446,.,+,.,ID=GFUI025974;biotype=protein_coding,GFUI025974,GFUI025974-PA,C,GO:0005634,nucleus,4550.224904,0.808055,5.725783,GFUI025974
4,Scaffold730,54979,54980,Scaffold730,VectorBase,gene,55178,57441,.,-,.,ID=GFUI046105;biotype=protein_coding,GFUI046105,GFUI046105-PA,P,GO:0006470,protein dephosphorylation,2043.008121,0.716502,11.013535,GFUI046105
5,Scaffold730,54979,54980,Scaffold730,VectorBase,gene,55178,57441,.,-,.,ID=GFUI046105;biotype=protein_coding,GFUI046105,GFUI046105-PA,P,GO:0035335,peptidyl-tyrosine dephosphorylation,2839.457697,0.379144,11.690944,GFUI046105
6,Scaffold730,54979,54980,Scaffold730,VectorBase,gene,55178,57441,.,-,.,ID=GFUI046105;biotype=protein_coding,GFUI046105,GFUI046105-PA,F,GO:0004725,protein tyrosine phosphatase activity,1190.337944,0.356301,8.510896,GFUI046105
7,Scaffold730,54979,54980,Scaffold730,VectorBase,gene,55178,57441,.,-,.,ID=GFUI046105;biotype=protein_coding,GFUI046105,GFUI046105-PA,F,GO:0016787,hydrolase activity,157.042081,0.793553,2.643033,GFUI046105
8,Scaffold730,54979,54980,Scaffold730,VectorBase,gene,55178,57441,.,-,.,ID=GFUI046105;biotype=protein_coding,GFUI046105,GFUI046105-PA,C,GO:0005737,cytoplasm,425.271068,0.521476,3.178476,GFUI046105
9,Scaffold87,733554,733555,Scaffold87,VectorBase,gene,728324,732701,.,+,.,ID=GFUI050274;biotype=protein_coding,GFUI050274,GFUI050274-PA,C,GO:0005789,endoplasmic reticulum membrane,974.111845,0.118899,9.060937,GFUI050274


# Specific questions asked in text so far:

## ...proximity to these SNPs using a window of 1000 bp returned X genes, including genes involved in X, Y, and X

## Answer set 1

In [217]:
def answer_set_1(df, title="None"):
    p = df.query("Aspect == 'P'")
    f = df.query("Aspect == 'F'")
    c = df.query("Aspect == 'C'")

    num_genes = len(df.gene_id.unique())
    num_contigs = len(df.bed3_seq.unique())
    num_func_annos = len(df.Name.unique())
    num_p = len(p.Name.unique())
    num_f = len(f.Name.unique())
    num_c = len(c.Name.unique())

    anno_stats = tbl(pd.DataFrame(df['Total Score'].describe()), headers=('Metric','Value'))
    anno_stats_p = tbl(pd.DataFrame(p['Total Score'].describe()), headers=('Metric','Value'))
    anno_stats_f = tbl(pd.DataFrame(f['Total Score'].describe()), headers=('Metric','Value'))
    anno_stats_c = tbl(pd.DataFrame(c['Total Score'].describe()), headers=('Metric','Value'))
    
    top10_annos = tbl(pd.DataFrame(df.Name.value_counts().head(10)), headers=('Annotation','Genes'))
    top10_annos_p= tbl(pd.DataFrame(p.Name.value_counts().head(10)), headers=('Annotation','Genes'))
    top10_annos_f= tbl(pd.DataFrame(f.Name.value_counts().head(10)), headers=('Annotation','Genes'))
    top10_annos_c= tbl(pd.DataFrame(c.Name.value_counts().head(10)), headers=('Annotation','Genes'))

    top10_annos_TS_800 = tbl(pd.DataFrame(df[df['Total Score'] >= 800].Name.value_counts().head(10)), headers=('Annotation','Genes'))
    top10_annos_TS_800_p = tbl(pd.DataFrame(p[p['Total Score'] >= 800].Name.value_counts().head(10)), headers=('Annotation','Genes'))
    top10_annos_TS_800_f = tbl(pd.DataFrame(f[f['Total Score'] >= 800].Name.value_counts().head(10)), headers=('Annotation','Genes'))
    top10_annos_TS_800_c = tbl(pd.DataFrame(c[c['Total Score'] >= 800].Name.value_counts().head(10)), headers=('Annotation','Genes'))


    return """
## {title} ##


- __Genes:__ {num_genes}
- __Contigs:__ {num_contigs}
- __Unique Terms (all):__ {num_func_annos}




### Annotation Scores ###

Table: All domains

{anno_stats}

Table: Process

{anno_stats_p}

Table: Function

{anno_stats_f}

Table: Cellular

{anno_stats_c}

----

### Top annotations ###

Table: All domains

{top10_annos}

Table: Process

{top10_annos_p}

Table: Function

{top10_annos_f}

Table: Cellular

{top10_annos_c}


----

### Top annotations (TS >= 800) ###

Table: All domains

{top10_annos_TS_800}

Table: Process

{top10_annos_TS_800_p}

Table: Function

{top10_annos_TS_800_f}

Table: Cellular

{top10_annos_TS_800_c}


\\newpage

""".format(
    title=title,
    num_genes=num_genes,
    num_contigs=num_contigs,
    num_func_annos=num_func_annos,
    anno_stats=anno_stats,
    anno_stats_p=anno_stats_p,
    anno_stats_f=anno_stats_f,
    anno_stats_c=anno_stats_c,
    top10_annos=top10_annos,
    top10_annos_p=top10_annos_p,
    top10_annos_f=top10_annos_f,
    top10_annos_c=top10_annos_c,
    top10_annos_TS_800=top10_annos_TS_800,
    top10_annos_TS_800_p=top10_annos_TS_800_p,
    top10_annos_TS_800_f=top10_annos_TS_800_f,
    top10_annos_TS_800_c=top10_annos_TS_800_c,
    )

### Example

In [155]:
Top10_PopPairwiseOverlap_Infection_win1000_argot = fr.Top10_PopPairwiseOverlap_Infection_win1000.merge(right=argot2_200, 
                                                      how='inner', 
                                                      on=None, 
                                                      left_on='gff3_rec', right_on='gene_id')

In [172]:
answer_set_1(Top10_PopPairwiseOverlap_Infection_win1000_argot)

Genes: 140
Contigs: 104
Unique Terms: 299


Annotation Scores
-----  ---------
count    912
mean    1491.44
std     2276.77
min      200.122
25%      343.728
50%      685.446
75%     1562.89
max    18243.2
-----  ---------

Top 10 annotations
Annotation                        Genes
------------------------------  -------
nucleus                              57
cytoplasm                            50
membrane                             31
integral component of membrane       21
hydrolase activity                   20
transferase activity                 19
metal ion binding                    18
mitochondrion                        17
DNA binding                          17
protein binding                      17

Top 10 annotations with TS >= 500
Annotation              Genes
--------------------  -------
nucleus                    44
cytoplasm                  28
transferase activity       14
membrane                   14
DNA binding                12
hydrolase activity         12
zi

### Run `answer_set_1()` for all data files

In [226]:
pdb

Automatic pdb calling has been turned ON


In [233]:
set1_results = []
for name,dset in fr.items():
    
    dset_merged = dset.merge(right=argot2_200, 
                            how='inner', 
                            on=None, 
                            left_on='gff3_rec', right_on='gene_id'
                            )

    set1_results.append(answer_set_1(dset_merged, title=name))
    
set1_results = '\n'.join(set1_results)

%clip set1_results

Copied to clipboard!


In [228]:
Selected_PopPairwiseMSvsNBOT_Environm_win500_search

,bed3_seq,bed3_start,bed3_end,gff3_seq,gff3_source,gff3_type,gff3_start,gff3_end,gff3_score,gff3_strand,gff3_phase,gff3_attributes,gff3_rec
0,Scaffold150,13770,13771,Scaffold150,VectorBase,gene,13476,16476,.,-,.,ID=GFUI009284;biotype=protein_coding,GFUI009284
1,Scaffold150,13771,13772,Scaffold150,VectorBase,gene,13476,16476,.,-,.,ID=GFUI009284;biotype=protein_coding,GFUI009284
